# Running

In [1]:
import cv2 # type: ignore
import mediapipe as mp # type: ignore
import time

cap = cv2.VideoCapture(0)


mpHands = mp.solutions.hands
hands = mpHands.Hands(max_num_hands=1, min_detection_confidence=0.7)
mpDraw = mp.solutions.drawing_utils

sTime = time.time()

pivot_x = 0
pivot_y = 0

row_list =[]

font = cv2.FONT_HERSHEY_SIMPLEX
org = (185, 50)
color = (0, 0, 255)

n = 5 #5 gestures

num = 1 #iter variable

while (cap.isOpened()):
    ret, img = cap.read()
    img = cv2.resize(img, (960, 540))
    x, y, c = img.shape
    t_elapsed = abs(sTime - time.time())
    if num>n:
        break
    print(t_elapsed,num)
    if t_elapsed==10:
        cv2.putText(img, 'Started', org, font, fontScale=1, thickness=2, color=color)
    if t_elapsed>10:
        cv2.putText(img, 'Recording for gesture {}'.format(num), org, font, fontScale=1, thickness=2, color=color)
    if (t_elapsed-10)//10==num:
        num += 1
        cv2.putText(img, 'Recording for gesture {}'.format(num), org, font, fontScale=1, thickness=2, color=color)
        time.sleep(3)
    cTime = time.time()

    imgRGB = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    results = hands.process(imgRGB)
    if results.multi_hand_landmarks!=0:
        if results.multi_hand_landmarks:
            for handlms in results.multi_hand_landmarks:
                for id,lms in enumerate(handlms.landmark):
                    #print(id,lms)
                    h,w,c = img.shape
                    #print(img.shape)
                    cx,cy = int(lms.x*w),int(lms.y*h)
                    #print("id:",id,", x:",cx,", y:",cy)
                    if id==0:
                        pivot_x = int(lms.x * x)
                        pivot_y = int(lms.y * y)
                        mpDraw.draw_landmarks(img, handlms, mpHands.HAND_CONNECTIONS)
                        if t_elapsed>10:
                            row_list.append(str(pivot_x))
                            row_list.append(str(pivot_y))
                    else:
                        lmx = int(lms.x * x)
                        lmy = int(lms.y * y)
                        mpDraw.draw_landmarks(img, handlms, mpHands.HAND_CONNECTIONS)
                        if t_elapsed>10:
                            row_list.append(str(pivot_x-lmx))
                            row_list.append(str(pivot_y-lmy))
                break
        if t_elapsed>10 and results.multi_hand_landmarks!=0:
            with open("landmarks_medium.csv", "a") as f:
                if len(row_list)>0:
                    f.write(",".join(row_list)+","+str(num-1)+"\n")
        row_list=[]

    if cv2.waitKey(1) & 0xFF == ord('q'):
        # if the 'q' is pressed quit.'OxFF' is for 64 bit.[if waitKey==True] is condition
        break
    if ret == True:
        cv2.imshow("result",img)
        pass
cap.release()
cv2.destroyAllWindows()

0.3739140033721924 1
0.4679117202758789 1
0.5104811191558838 1
0.5521981716156006 1
0.5912919044494629 1
0.6303164958953857 1
0.6745095252990723 1
0.720322847366333 1
0.7676379680633545 1
0.8078711032867432 1
0.8533675670623779 1
0.8976564407348633 1
0.9392566680908203 1
0.9993395805358887 1
1.0366814136505127 1
1.0943818092346191 1
1.132373332977295 1
1.171844720840454 1
1.2094812393188477 1
1.248553991317749 1
1.2942116260528564 1
1.3358125686645508 1
1.373603105545044 1
1.4150137901306152 1
1.476827621459961 1
1.5352671146392822 1
1.5700044631958008 1
1.6288683414459229 1
1.6679174900054932 1
1.719672441482544 1
1.7526910305023193 1
1.7924385070800781 1
1.8414134979248047 1
1.8963847160339355 1
1.9301605224609375 1
1.9620485305786133 1
1.999098777770996 1
2.033764600753784 1
2.06652569770813 1
2.107870578765869 1
2.150346040725708 1
2.1940431594848633 1
2.2578542232513428 1
2.3033993244171143 1
2.3669776916503906 1
2.413938522338867 1
2.4614102840423584 1
2.505918025970459 1
2.55460

# Training

In [ ]:
import pandas as pd # type: ignore
import numpy as np # type: ignore

df = pd.read_csv('landmarks_medium.csv')
#df.head()
#df.info()

from sklearn.model_selection import train_test_split # type: ignore
x_train,x_test,y_train,y_test = train_test_split(df.drop('class_id',axis=1),df['class_id'],
                                                 test_size=0.30,random_state=1)

from sklearn.linear_model import LogisticRegression # type: ignore

logmodel = LogisticRegression()
logmodel.fit(x_train,y_train)

prediction = logmodel.predict(x_test)

#For testing accuracy
from sklearn.metrics import confusion_matrix # type: ignore
from sklearn.metrics import accuracy_score # type: ignore

accuracy_m = confusion_matrix(y_test,prediction)
accuracy_s = accuracy_score(y_test,prediction)

#saving the model using pickle
import pickle

filename = 'gesture_model.sav'
pickle.dump(logmodel, open(filename, 'wb'))

# RT Implementation

In [ ]:
import cv2 # type: ignore
import mediapipe as mp # type: ignore
import time
import pickle

cap = cv2.VideoCapture(0)


mpHands = mp.solutions.hands
hands = mpHands.Hands(max_num_hands=1, min_detection_confidence=0.7)
mpDraw = mp.solutions.drawing_utils

sTime = time.time()

pivot_x = 0
pivot_y = 0

row_list =[]

font = cv2.FONT_HERSHEY_SIMPLEX
org = (185, 50)
color = (0, 0, 255)

model_file_name = 'gesture_model.sav'
model = pickle.load(open(model_file_name, 'rb'))
mpHands = mp.solutions.hands
hands = mpHands.Hands(max_num_hands=1, min_detection_confidence=0.7)
mpDraw = mp.solutions.drawing_utils
classID = 0
while (cap.isOpened()):
    ret, img = cap.read()
    img = cv2.resize(img, (960, 540))
    x, y, c = img.shape
    t_elapsed = abs(sTime - time.time())

    if t_elapsed>60:
        break
    cTime = time.time()

    imgRGB = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    results = hands.process(imgRGB)
    if results.multi_hand_landmarks!=0:
        if results.multi_hand_landmarks:
            for handlms in results.multi_hand_landmarks:
                for id,lms in enumerate(handlms.landmark):
                    #print(id,lms)
                    h,w,c = img.shape
                    #print(img.shape)
                    cx,cy = int(lms.x*w),int(lms.y*h)
                    #print("id:",id,", x:",cx,", y:",cy)
                    if id == 0:
                        pivot_x = int(lms.x * x)
                        pivot_y = int(lms.y * y)
                        mpDraw.draw_landmarks(img, handlms, mpHands.HAND_CONNECTIONS)
                        row_list.append(pivot_x)
                        row_list.append(pivot_y)
                    else:
                        lmx = int(lms.x * x)
                        lmy = int(lms.y * y)
                        mpDraw.draw_landmarks(img, handlms, mpHands.HAND_CONNECTIONS)
                        row_list.append(pivot_x - lmx)
                        row_list.append(pivot_y - lmy)
                break
            prediction = model.predict([row_list])
            cv2.putText(img, 'Predicted gesture {}'.format(prediction[0]), org, font, fontScale=1, thickness=2, color=color)
        row_list=[]

    if cv2.waitKey(1) & 0xFF == ord('q'):
        # if the 'q' is pressed quit.'OxFF' is for 64 bit.[if waitKey==True] is condition
        break
    if ret == True:
        cv2.imshow("result",img)
        pass
cap.release()
cv2.destroyAllWindows()